In [1]:
import pandas as pd
import pickle
import os

print("--- ЭТАП 5: ПОДГОТОВКА ОТЧЕТА (INFERENCE) ---")

# 1. Загружаем именно ту модель, которую обучили в шаге 03
model_filename = 'model_pro.pkl'

if not os.path.exists(model_filename):
    print(f"❌ ОШИБКА: Файл {model_filename} не найден!")
    print("👉 Сначала запустите ноутбук '03_Training_PRO.ipynb', чтобы создать модель.")
else:
    print(f"⏳ Загружаем {model_filename}...")
    
    # Используем pickle, так как мы сохраняли Pipeline (Sklearn)
    with open(model_filename, 'rb') as file:
        pipe = pickle.load(file)
    print("✅ Модель успешно загружена!")

    # 2. Загружаем подготовленные признаки
    # Важно: используем тот же файл, что и при обучении
    features_file = 'features_pro.parquet'
    
    if not os.path.exists(features_file):
        print(f"❌ ОШИБКА: Не найден файл с признаками {features_file}")
    else:
        print("⏳ Загружаем данные для прогноза...")
        df = pd.read_parquet(features_file)

        # Сохраняем ID для отчета
        ids = df['id']
        
        # Убираем лишние колонки (если они есть), чтобы оставить только признаки
        # Модель сама знает, как их масштабировать, так как это Pipeline
        X = df.drop(columns=['id', 'flag'], errors='ignore')

        # 3. Предсказываем
        print("🔮 Робот делает предсказания...")
        # predict_proba возвращает [вероятность_0, вероятность_1]
        # Нам нужна вторая колонка (индекс 1) - вероятность дефолта
        preds_proba = pipe.predict_proba(X)[:, 1]

        # 4. Формируем таблицу для сдачи
        submission = pd.DataFrame({
            'id': ids,
            'score': preds_proba
        })

        print(f"Пример результата:\n{submission.head()}")

        # 5. Сохраняем итоговый файл
        submission.to_csv('submission_final.csv', index=False)
        print("\n🎉 ГОТОВО! Файл 'submission_final.csv' создан.")

--- ЭТАП 5: ПОДГОТОВКА ОТЧЕТА (INFERENCE) ---
⏳ Загружаем model_pro.pkl...
✅ Модель успешно загружена!
⏳ Загружаем данные для прогноза...
🔮 Робот делает предсказания...
Пример результата:
   id     score
0   0  0.285691
1   1  0.596103
2   2  0.533591
3   3  0.604469
4   4  0.596991

🎉 ГОТОВО! Файл 'submission_final.csv' создан.
